In [ ]:
!pip install h2o

     |████████████████████████████████| 177.0 MB 15 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=4f6669bf658b4474baeac247f668ce0f93352ed3f2a209527d8138de0f498d36
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o


In [ ]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [ ]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmptqjgcu55
  JVM stdout: /tmp/tmptqjgcu55/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmptqjgcu55/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_unknownUser_2pr68e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [ ]:
test, train = dataSetup(41163)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
train.shape

(7500, 2001)

In [ ]:
x = train.columns
y = "class"
x.remove(y)

In [ ]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid_1_AutoML_1_20220502_215808_model_3


Model Summary: 


,,number_of_trees
0,,30.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.5862815748893927
RMSE: 0.7656902604117364
LogLoss: 1.4679833476728195
Mean Per-Class Error: 0.5577187976675446
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,Error,Rate
0,417.0,89.0,233.0,111.0,207.0,0.605487,"640 / 1,057"
1,137.0,349.0,208.0,105.0,190.0,0.647118,640 / 989
2,111.0,61.0,619.0,110.0,197.0,0.436248,"479 / 1,098"
3,149.0,80.0,238.0,381.0,189.0,0.632594,"656 / 1,037"
4,104.0,74.0,239.0,102.0,592.0,0.467147,"519 / 1,111"
5,918.0,653.0,1537.0,809.0,1375.0,0.554422,"2,934 / 5,292"



Top-5 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc,validation_rmse,validation_logloss,validation_classification_error,validation_auc,validation_pr_auc
0,,2022-05-02 22:42:02,3 min 35.575 sec,0.0,0.800000,1.609438,0.804044,NaN,NaN,0.800000,1.609438,0.814016,NaN,NaN
1,,2022-05-02 22:42:10,3 min 43.729 sec,5.0,0.792902,1.577870,0.702192,NaN,NaN,0.800137,1.612972,0.797844,NaN,NaN
2,,2022-05-02 22:42:17,3 min 50.867 sec,10.0,0.787180,1.553763,0.655518,NaN,NaN,0.800911,1.619482,0.803235,NaN,NaN
3,,2022-05-02 22:42:25,3 min 57.981 sec,15.0,0.781326,1.529794,0.623394,NaN,NaN,0.800961,1.622549,0.787062,NaN,NaN
4,,2022-05-02 22:42:32,4 min 5.141 sec,20.0,0.776124,1.508955,0.600151,NaN,NaN,0.801088,1.626173,0.787062,NaN,NaN
5,,2022-05-02 22:42:39,4 min 12.463 sec,25.0,0.770640,1.487262,0.575964,NaN,NaN,0.800726,1.627440,0.787062,NaN,NaN
6,,2022-05-02 22:42:46,4 min 19.664 sec,30.0,0.765690,1.467983,0.554422,NaN,NaN,0.801301,1.633263,0.773585,NaN,NaN



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,V1651,32.162807,1.000000,0.013522
1,V1949,24.060635,0.748089,0.010116
2,V475,23.454281,0.729236,0.009861
3,V853,21.106997,0.656255,0.008874
4,V1875,20.270651,0.630251,0.008522
5,V1071,20.244902,0.629451,0.008512
6,V1974,19.010757,0.591079,0.007993
7,V1682,18.923735,0.588373,0.007956
8,V1695,18.801867,0.584584,0.007905
9,V1010,18.670643,0.580504,0.007850



See the whole table with table.as_data_frame()


In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_grid_1_AutoML_1_20220502_215808_model_3,0.772672,1.63326,0.801301,0.642084
GBM_3_AutoML_1_20220502_215808,0.77722,1.62211,0.798135,0.637019
XGBoost_2_AutoML_1_20220502_215808,0.778744,1.68832,0.799657,0.639452
GBM_2_AutoML_1_20220502_215808,0.780456,1.63521,0.800972,0.641556
GBM_grid_1_AutoML_1_20220502_215808_model_1,0.782278,1.63261,0.799571,0.639314
XRT_1_AutoML_1_20220502_215808,0.782494,1.67958,0.802699,0.644325
XGBoost_1_AutoML_1_20220502_215808,0.784239,1.72698,0.805346,0.648583
GBM_grid_1_AutoML_1_20220502_215808_model_2,0.784316,1.65199,0.800404,0.640646
XGBoost_grid_1_AutoML_1_20220502_215808_model_2,0.784819,1.66241,0.801489,0.642385
GBM_4_AutoML_1_20220502_215808,0.786523,1.64114,0.800312,0.640499
